In [1]:
from gen3.auth import Gen3Auth
#NOTE: using the Gen3File object to get presigned url so no need for these 
# (previously used to call directly from fence API)
#from urllib.parse import parse_qsl, urlencode, urlparse, urlunparse
from gen3.file import Gen3File
from gen3.index import Gen3Index
from gen3.submission import Gen3Submission
import os
import requests
import hashlib

In [2]:
COMMONS = 'https://jcoin.datacommons.io/'
CREDS_PATH = 'credentials.json'
AWS_BUCKET = "s3://jcoinprod-default-258867494168-upload"


In [3]:
program = "JCOIN"
project = "Chestnut" #project as it appears in fence (ie profile)
uploader = "" #uploader identifier (ie email)

In [4]:
auth = Gen3Auth(COMMONS,CREDS_PATH)
index = Gen3Index(auth)
sub = Gen3Submission(auth)

## Prototype code for one file upload

Current gen3 file information 
(this includes current_object_id of core measures mapped to sheepdog and 
the indexed info of this sheepdog record)

In [5]:
for record in sub.export_node(program,project,"reference_file",fileformat='json')['data']:
    if f"core-measures" in record.get("submitter_id",""):
        current_record = record.copy()

In [6]:
current_object_id = current_record['object_id']
file_name = current_record['file_name']

current_index = index.get_record(current_object_id)

Local file information

In [7]:
file_path = f"tmp/{file_name}"
with open(file_path, 'rb') as f:
    data = f.read()
    
# get file info
new_md5sum = hashlib.md5(data).hexdigest()
new_filesize = os.path.getsize(file_path)

In [8]:
#generate new version num
if current_index['version']:
    #make this a command (eg indicating which of the 3 version enhancements it is)
    pass
else:
    new_version_num = '.'.join(["0","0","1"]) #form of incremental version change code in above if statemnt 


```python
current_index

{'acl': [],
 'authz': ['/programs/JCOIN/projects/TEST'],
 'baseid': 'abdasfdsafdsfdsf',
 'created_date': '2099-12-07T21:32:49.549732',
 'did': 'dg.6VTS/dasfasdfdf',
 'file_name': None,
 'form': 'object',
 'hashes': {'md5': '7a259dea963796bc678c457356c82d14'},
 'metadata': {},
 'rev': '62ee5b42',
 'size': 850335,
 'updated_date': '2022-12-07T21:33:02.764339',
 'uploader': None,
 'urls': ['s3://jcoinprod-default-258867494168-upload/dg.6VTS/dasfasdfdf/core-measures-test.zip'],
 'urls_metadata': {'s3://jcoinprod-default-258867494168-upload/dg.6VTS/dasfasdfdf/core-measures-test.zip': {}},
 'version': None}
 ```

Check if new version locally

In [9]:
#TODO: check if new version of file
is_new_version = new_md5sum!=current_index['hashes']['md5']

In [10]:
is_new_version

True

Mint new GUID

In [11]:
new_object_id = requests.get(f"{COMMONS}/index/guid/mint?count=1").json()["guids"][0]

Create new indexd record

In [12]:
# update new record more index metadata
#TODO: add version string
default_authz = '/programs/{}/projects/{}'.format(program, project)
authz= [default_authz]

new_index = index.create_new_version(
    guid=current_object_id,
    hashes={'md5':new_md5sum},
    size=new_filesize,
    did=new_object_id, 
    authz=authz
)

Upload new version of file

In [13]:
#get presigned url to upload
gen3files = Gen3File(auth)
new_file_presign = gen3files.upload_file_to_guid(new_object_id, file_name, expires_in=3600)
new_file_url = new_file_presign['url']
headers = {
    "Content-Type":"application/binary",
}
upload = requests.put(new_file_url, data=data, headers=headers)
upload.raise_for_status()

In [14]:
file_url = f"{AWS_BUCKET}/{new_object_id}/{file_name}"
output = index.update_record(
    new_object_id,
    urls=[file_url],
    # urls_metadata={file_url:{}}
)

```python
output

{'acl': [],
 'authz': ['/programs/JCOIN/projects/TEST'],
 'baseid': 'adsfdsf',
 'created_date': '2099-03-17T02:21:05.818203',
 'did': 'dg.6VTS/asdfdsfdsfsdf',
 'file_name': None,
 'form': 'object',
 'hashes': {'md5': 'dsfasdf'},
 'metadata': {},
 'rev': '5ae5d479',
 'size': 766615,
 'updated_date': '2023-03-17T02:21:22.387864',
 'uploader': None,
 'urls': ['s3://jcoinprod-default-258867494168-upload/dg.6VTS/asdfdsfdsfsdf/core-measures-ytest.zip'],
 'urls_metadata': {'s3://jcoinprod-default-258867494168-upload/dg.6VTS/asdfdsfdsfsdf/core-measures-test.zip': {}},
 'version': None}
 ```

Map to sheepdog

In [15]:
  sheepdog_rec = {
    "core_metadata_collections": [
      {
        "submitter_id": f"core-measures-{project}"
      }
  ],
  "type": "reference_file",
  "project_id": f"{program}-{project}",
  "submitter_id": f"core-measures-{project}",
  "data_category": "Core Measures",
  "data_format": "ZIP",
  "data_type": "Interview",
  "md5sum":new_md5sum,
  "file_size":new_filesize,
  "file_name":file_name,
  "object_id":new_object_id,
}

In [16]:
output = sub.submit_record(program,project,json=sheepdog_rec)

[2023-03-16 21:35:38,974][   INFO] 
Using the Sheepdog API URL https://jcoin.datacommons.io/api/v0/submission/JCOIN/Chestnut



```python
output

{'code': 200,
 'created_entity_count': 0,
 'entities': [{'action': 'update',
   'errors': [],
   'id': 'abcasdfdsfsdfasdfsd',
   'type': 'reference_file',
   'unique_keys': [{'project_id': 'JCOIN-TEST',
     'submitter_id': 'core-measures-TEST'}],
   'valid': True,
   'warnings': []}],
 'entity_error_count': 0,
 'message': 'Transaction successful.',
 'success': True,
 'transaction_id': 9603,
 'transactional_error_count': 0,
 'transactional_errors': [],
 'updated_entity_count': 1}
 ```